# 06. PyTorch Transfer Learning

Let's take a well performing pre-trained model and adjust it to one of our own problems.

We can increase the performance of our model by getting a help from other already trained good models.

**What we're going to cover**
- take a pretrained model from `torchvision.models`
- and customize it to work on (and hopefully improve) our `foodvision` problem

Steps that we will do:
1. Setting up
2. Get data
3. Create datasets ad dataloaders
4. Get and Customize a pretrained model
5. Train model
6. Evaluate the model by plotting loss curves
7. Make predictions on images from the test set

## 1. Setting up

In [16]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split('.')[0]) >=2, 'torch version should be 2.0+'
    assert int(torchvision.__version__.split('.')[1]) >=17, 'torchvision version should be 0.17+'
    print(f'torch version: {torch.__version__}')
    print(f'torchvision version: {torchvision.__version__}')
except:
    print('[INFO] torch/torchvision verson not as required, installing nightly versions')
    !pip install -U torch torchvision
    import torch
    import torchvision
    print(f'torch version: {torch.__version__}')
    print(f'torchvision version: {torchvision.__version__}')
    

torch version: 2.2.2
torchvision version: 0.17.2


In [19]:
import matplotlib.pyplot as plot
import torch
import torchvision
from torch import nn
from torchvision import transforms

# try torchinfo, install if not installed
try:
    from torchinfo import summary
except:
    print(f"[INFO] Couldn't find torchinfo, intalling it.")
    !pip install -q torchinfo
    print(f'installed torchinfo {torchinfo.__version__}')
    from torchinfo import summary

## setup device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

## 2. Getting Data

before we start transfer learning, we need a dataset.


In [23]:
from pathlib import Path
from zipfile import ZipFile
import requests
import os

data_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"

data_path = Path('data')
image_data_path = data_path / 'pizza_steak_sushi'

# create folders if not already available
if image_data_path.is_dir():
    print('folders already eists')
else:
    print("Folders doesn't exist, creating one..")
    image_data_path.mkdir(parents=True, exist_ok=True)

    # download data
    with open(data_path / 'pizza_steak_sushi.zip', 'wb') as f:
        request = requests.get(data_url)
        print(f'Downloading data...')
        f.write(request.content)
        print(f'Download done.')
    # unzip
    with ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_file:
        print(f'Unzipping...')
        zip_file.extractall(image_data_path)
        print('Extracted..')
    # remove zip file
    os.remove(data_path/'pizza_steak_sushi.zip')


Folders doesn't exist, creating one..
Download done.
Unzipping...


## 3. Create datasets and dataloader
now that we have downloaded the data we should create `datasets` and corresponding `dataloader`

### 3.1 Creating a transform for torchvision.models (manual = old method)

since we'll be using a pretrained model from `torchvision.models`, there's a specific transform we need to prepare our images first.

When using a pretrained model, it's important that **your custom data going into the model is prepared in the same way as the original training data that went into the model**

1. All pretrained models expect input images normalized in the same way, i.e. `mini batches` of 3-channel images of shape (3 x H x W) where H and W are expected to be at least 224.
    - But some models might expect H and W to be of other sizes. We have to check the documentation for that.
2. Images have to be loaded in to a range of [0,1[
3. then normalized using `mean = [0.485, 0.456, 0.406]` and `std = [0.229, 0.224, 0.225]`

we can achieve this in the following way.

In [24]:
manual_transform = transforms.Compose([
    transforms.Resize(size=(224,224)), # mini batchs of shape 3,224,224
    transforms.ToTensor(), # turn image values to between 0, 1
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std = [0.229, 0.224, 0.225]
    )
])

In [25]:
train_data_path = data_path / 'pizza_steak_sushi'/ 'train'
test_data_path = data_path / 'pizza_steak_sushi'/ 'test'

train_data_path, test_data_path

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [29]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path,
                                              transform=manual_transform,
                                              target_transform=None)
test_data = torchvision.datasets.ImageFolder(root=test_data_path,
                                              transform=manual_transform,
                                              target_transform=None)

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
train_dataloader = torch.utils.data.DataLoader(dataset=train_data,
                                               batch_size=BATCH_SIZE,
                                               num_workers=NUM_WORKERS,
                                               shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data,
                                               batch_size=BATCH_SIZE,
                                               num_workers=NUM_WORKERS,
                                               shuffle=False)
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

### 3.2 Creating a transform for `torchvision.models` - new method

As of `torchvision v0.13+`, an automatic transform creation feature has been added.

When you setup a model from `torchvision.models` 

select a pretrained model weights you'd like to use

for example:

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

where,
- `EfficientNet_B0_Weights` is the model architecture weights we'd like to use
- `DEFAULT` means the best available weights (the best performance in ImageNet)
  - Depending on the model architecture we can also see 'IMAGENET_V1` and `IMAGENET_V2`, where generally the higher version number is better.
  - `DEFAULT` is easiest option to choose.

let's try it out

In [31]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

Now we have access to the transform associated with the `EfficientNet_BO_Weights model`

In [33]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

The benefit of using auto transform is that, it `comes` with the `model architecture` we chose to use.

we don't have to create it by hand.